In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2

## Load the training data into feature matrix, class labels, and event ids:

In [14]:
from proj1_helpers import *
import implementations as imp
%run pre_processing.ipynb

In [3]:
train_path = '../resources/train.csv'
test_path = '../resources/test.csv'

In [4]:
(tx_1,y_1,ids_1), (tx_2,y_2,ids_2), (tx_3,y_3,ids_3),indexes,parameters = process_data(train_path)

In [5]:
(xtest_1,ids_1), (xtest_2,ids_2), (xtest_3,ids_3) = process_test(test_path,indexes,parameters)

## Do your thing crazy machine learning thing here :) ...

## Cross validation helper functions

In [6]:
def build_k_indices(y, k_fold, seed):
    """build k indices for k-fold."""
    num_row = y.shape[0]
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval]
                 for k in range(k_fold)]
    return np.array(k_indices)

In [7]:
def compute_accuracy(y,x,w):
    y_pred = predict_labels(w,x)
    return (y_pred==y).mean()

In [72]:
from random import seed
from random import randint
def evaluate_stability(y,tx,lambda_,degree):
    acc_train = []
    acc_test = []  
    k_fold = 5

    for i in range (40):
        seed = randint(1, 40)
        k_indices = build_k_indices(y, k_fold, seed)
        (ltr,lte,acctr,accte) = cross_validation_ls_sgd(y, tx, k_indices, k_fold, degree)
        #(ltr,lte,acctr,accte) = cross_validation_rg(y, tx, k_indices, k_fold, lambda_, degree)
        #(ltr,lte,acctr,accte) = cross_validation_ls(y, tx, k_indices, k_fold, degree)
        acc_train.append(acctr)
        acc_test.append(accte)
    
    return (np.mean(acc_test),np.std(acc_test))

In [ ]:
## Best parameters with least squares gradient descent 

In [ ]:
#cross validation for least squares gradient descent
def cross_validation_ls_sgd(y, tx, k_indices, k_fold , degree):
    gamma = 0.01
    max_iters = 2 
    loss_te = np.array([])
    loss_tr = np.array([])
    acc_te = np.array([]) 
    acc_tr = np.array([]) 
    #print(x.shape)
    for i in range(k_fold):
        sub = np.delete(k_indices,i,axis=0).flatten()
        x_train = tx[sub]
        y_train = y[sub]
        x_test = tx[k_indices[i]]
        y_test = y[k_indices[i]]
        
        x_train = expand_with_pairwise_products(x_train,degree)
        x_test = expand_with_pairwise_products(x_test,degree)
        initial_w = np.zeros((x_train.shape[1], 1))
        
        #x_train = np.append(x_train,np.cos(x_train),axis=1)
        #x_train = np.append(x_train,np.sin(x_train),axis=1)
        #x_test = np.append(x_test,np.cos(x_test),axis=1)
        #x_test = np.append(x_test,np.sin(x_test),axis=1)

        w,loss_train = imp.least_squares_GD(y_train, x_train,initial_w, max_iters, gamma)
        acc_train = compute_accuracy(y_train,x_train,w)
        acc_test = compute_accuracy(y_test,x_test,w)
        loss_test = imp.compute_loss(y_test, x_test, w)
        loss_tr = np.append(loss_tr,loss_train)
        loss_te= np.append(loss_te,loss_test)
        acc_tr = np.append(acc_tr,acc_train)
        acc_te = np.append(acc_te,acc_test)
    
    return np.mean(loss_tr),np.mean(loss_te),np.mean(acc_tr), np.mean(acc_te)

In [ ]:
for deg in range (0,5):
    meanacc,stdacc = evaluate_stability(y_1,tx_1,1,deg)
    print("for deg ", deg, " : mean acc = ", meanacc, " std acc = ",stdacc) 

In [ ]:
for deg in range (0,5):
    meanacc,stdacc = evalutate_stability(y_2,tx_2,1,deg)
    print("for deg ", deg, " : mean acc = ", meanacc, " std acc = ",stdacc)

In [ ]:
for deg in range (0,5):
    meanacc,stdacc = evalutate_stability(y_3,tx_3,1,deg)
    print("for deg ", deg, " : mean acc = ", meanacc, " std acc = ",stdacc)

In [ ]:
#Best results for least squares stochastic gradient descent 
#batch 1 : deg = 8   avg acc =    std acc = 
#batch 2 : deg = 12  avg acc =    std acc = 
#batch 3 : deg = 12  avg acc =    std acc = 
#overall avc acc = 

## Best parameters with least squares stochastic gradient descent

In [78]:
#cross validation for least squares stochastic gradient descent
def cross_validation_ls_sgd(y, tx, k_indices, k_fold , degree):
    gamma = 0.01
    max_iters = 2
    batch_size = 100 
    loss_te = np.array([])
    loss_tr = np.array([])
    acc_te = np.array([]) 
    acc_tr = np.array([]) 
    #print(x.shape)
    for i in range(k_fold):
        sub = np.delete(k_indices,i,axis=0).flatten()
        x_train = tx[sub]
        y_train = y[sub]
        x_test = tx[k_indices[i]]
        y_test = y[k_indices[i]]
        
        x_train = expand_with_pairwise_products(x_train,degree)
        x_test = expand_with_pairwise_products(x_test,degree)
        initial_w = np.zeros((x_train.shape[1], 1))
        
        #x_train = np.append(x_train,np.cos(x_train),axis=1)
        #x_train = np.append(x_train,np.sin(x_train),axis=1)
        #x_test = np.append(x_test,np.cos(x_test),axis=1)
        #x_test = np.append(x_test,np.sin(x_test),axis=1)

        w,loss_train = imp.least_squares_SGD(y_train, x_train,initial_w, batch_size, max_iters, gamma)
        acc_train = compute_accuracy(y_train,x_train,w)
        acc_test = compute_accuracy(y_test,x_test,w)
        loss_test = imp.compute_loss(y_test, x_test, w)
        loss_tr = np.append(loss_tr,loss_train)
        loss_te= np.append(loss_te,loss_test)
        acc_tr = np.append(acc_tr,acc_train)
        acc_te = np.append(acc_te,acc_test)
    
    return np.mean(loss_tr),np.mean(loss_te),np.mean(acc_tr), np.mean(acc_te)

In [ ]:
for deg in range (0,5):
    meanacc,stdacc = evaluate_stability(y_1,tx_1,1,deg)
    print("for deg ", deg, " : mean acc = ", meanacc, " std acc = ",stdacc)  

for deg  0  : mean acc =  0.7366177059353418  std acc =  0.009624198161589994


In [ ]:
for deg in range (0,5):
    meanacc,stdacc = evalutate_stability(y_2,tx_2,1,deg)
    print("for deg ", deg, " : mean acc = ", meanacc, " std acc = ",stdacc)

In [ ]:
for deg in range (0,5):
    meanacc,stdacc = evalutate_stability(y_3,tx_3,1,deg)
    print("for deg ", deg, " : mean acc = ", meanacc, " std acc = ",stdacc)

In [ ]:
#Best results for least squares stochastic gradient descent 
#batch 1 : deg = 8   avg acc =    std acc = 
#batch 2 : deg = 12  avg acc =    std acc = 
#batch 3 : deg = 12  avg acc =    std acc = 
#overall avc acc = 

## Best parameters with least squares

In [85]:
#cross validation for least squares
def cross_validation_ls(y, x, k_indices, k_fold , degree):
    loss_te = np.array([])
    loss_tr = np.array([])
    acc_te = np.array([]) 
    acc_tr = np.array([]) 
    #print(x.shape)
    for i in range(k_fold):
        sub = np.delete(k_indices,i,axis=0).flatten()
        x_train = x[sub]
        y_train = y[sub]
        x_test = x[k_indices[i]]
        y_test = y[k_indices[i]]
        
        x_train = expand_with_pairwise_products(x_train,degree)
        x_test = expand_with_pairwise_products(x_test,degree)
        
        #x_train = np.append(x_train,np.cos(x_train),axis=1)
        #x_train = np.append(x_train,np.sin(x_train),axis=1)
        #x_test = np.append(x_test,np.cos(x_test),axis=1)
        #x_test = np.append(x_test,np.sin(x_test),axis=1)

        w,loss_train = imp.least_squares(y_train, x_train)
        acc_train = compute_accuracy(y_train,x_train,w)
        acc_test = compute_accuracy(y_test,x_test,w)
        loss_test = imp.compute_loss(y_test, x_test, w)
        loss_tr = np.append(loss_tr,loss_train)
        loss_te= np.append(loss_te,loss_test)
        acc_tr = np.append(acc_tr,acc_train)
        acc_te = np.append(acc_te,acc_test)
    
    return np.mean(loss_tr),np.mean(loss_te),np.mean(acc_tr), np.mean(acc_te)

In [86]:
for deg in range (0,14):
    meanacc,stdacc = evalutate_stability(y_1,tx_1,1,deg)
    print("for deg ", deg, " : mean acc = ", meanacc, " std acc = ",stdacc)   

for deg  0  : mean acc =  0.7957191472325092  std acc =  0.00013912521269141684
for deg  1  : mean acc =  0.8323631268141328  std acc =  0.00015279399169057867
for deg  2  : mean acc =  0.8337558802922631  std acc =  0.00012203090884993148
for deg  3  : mean acc =  0.8357466720048041  std acc =  0.0001682429115639703
for deg  4  : mean acc =  0.8387889100190172  std acc =  0.00014135044294997072
for deg  5  : mean acc =  0.8409643679311379  std acc =  0.00013730502370455237
for deg  6  : mean acc =  0.8381423280952858  std acc =  0.014848742056491664
for deg  7  : mean acc =  0.8398308477629867  std acc =  0.011566748377691465
for deg  8  : mean acc =  0.8450440396356722  std acc =  0.0003718353247391593
for deg  9  : mean acc =  0.8415108597737964  std acc =  0.009320020138757627
for deg  10  : mean acc =  0.8441447302572316  std acc =  0.002260838641793926
for deg  11  : mean acc =  0.7880307276548894  std acc =  0.03329041646415769
for deg  12  : mean acc =  0.7738524672204984  std 

In [87]:
for deg in range (0,14):
    meanacc,stdacc = evalutate_stability(y_2,tx_2,1,deg)
    print("for deg ", deg, " : mean acc = ", meanacc, " std acc = ",stdacc)

for deg  0  : mean acc =  0.6699277792107299  std acc =  0.0004526787753867319
for deg  1  : mean acc =  0.7513373742584474  std acc =  0.0003422599879055013
for deg  2  : mean acc =  0.7724258447253033  std acc =  0.00018265385494497166
for deg  3  : mean acc =  0.77662883672943  std acc =  0.00028023269240217346
for deg  4  : mean acc =  0.7841842919783337  std acc =  0.00020583163317026163
for deg  5  : mean acc =  0.7855384317771474  std acc =  0.0002391357508320971
for deg  6  : mean acc =  0.7866397988135156  std acc =  0.00029886878150629776
for deg  7  : mean acc =  0.7933460149600207  std acc =  0.0002643474316164753
for deg  8  : mean acc =  0.8006686871292235  std acc =  0.00024931690743162536
for deg  9  : mean acc =  0.8040624193964406  std acc =  0.0001953734512193018
for deg  10  : mean acc =  0.8041726850657724  std acc =  0.00020360554846343565
for deg  11  : mean acc =  0.8047517410368842  std acc =  0.00029961911925489556
for deg  12  : mean acc =  0.8062819190095436

In [88]:
for deg in range (0,14):
    meanacc,stdacc = evalutate_stability(y_3,tx_3,1,deg)
    print("for deg ", deg, " : mean acc = ", meanacc, " std acc = ",stdacc)

for deg  0  : mean acc =  0.7311421284808381  std acc =  0.0004536973572947324
for deg  1  : mean acc =  0.7868341604631928  std acc =  0.0003002731708084146
for deg  2  : mean acc =  0.7914826302729528  std acc =  0.00041324946611870876
for deg  3  : mean acc =  0.8055459057071961  std acc =  0.0003395852461630336
for deg  4  : mean acc =  0.8128522194651226  std acc =  0.0003254983411505178
for deg  5  : mean acc =  0.8147077474496831  std acc =  0.00031769216976191905
for deg  6  : mean acc =  0.8159043286462643  std acc =  0.0003650190492085563
for deg  7  : mean acc =  0.8233478081058724  std acc =  0.00033807580422777153
for deg  8  : mean acc =  0.8324965536255858  std acc =  0.00026080106775609464
for deg  9  : mean acc =  0.8359753239591949  std acc =  0.00030998293922770574
for deg  10  : mean acc =  0.8360366694237662  std acc =  0.0003029957051469666
for deg  11  : mean acc =  0.8368927488282327  std acc =  0.00038824229970597744
for deg  12  : mean acc =  0.834314860766473

In [ ]:
#Best results for least squares
#batch 1 : deg = 8   avg acc = 0.8450440396356722   std acc = 0.0003718353247391593
#batch 2 : deg = 12  avg acc = 0.8062819190095436   std acc = 0.00014765868113824493
#batch 3 : deg = 12  avg acc = 0.8368927488282327   std acc = 0.009791875569386417

## Best parameters with ridge regression

In [25]:
def cross_validation_rg(y, x, k_indices, k , lambda_, degree):
    loss_te = np.array([])
    loss_tr = np.array([])
    acc_te = np.array([]) 
    acc_tr = np.array([]) 
    #print(x.shape)
    for i in range(k):
        sub = np.delete(k_indices,i,axis=0).flatten()
        x_train = x[sub]
        y_train = y[sub]
        x_test = x[k_indices[i]]
        y_test = y[k_indices[i]]
        
        x_train = expand_with_pairwise_products(x_train,degree)
        x_test = expand_with_pairwise_products(x_test,degree)
        
        #x_train = np.append(x_train,np.cos(x_train),axis=1)
        #x_train = np.append(x_train,np.sin(x_train),axis=1)
        #x_test = np.append(x_test,np.cos(x_test),axis=1)
        #x_test = np.append(x_test,np.sin(x_test),axis=1)

        w,loss_train = imp.ridge_regression(y_train, x_train,lambda_)
        acc_train = compute_accuracy(y_train,x_train,w)
        acc_test = compute_accuracy(y_test,x_test,w)
        loss_test = imp.compute_loss(y_test, x_test, w)
        loss_tr = np.append(loss_tr,loss_train)
        loss_te= np.append(loss_te,loss_test)
        acc_tr = np.append(acc_tr,acc_train)
        acc_te = np.append(acc_te,acc_test)
    
    return np.mean(loss_tr),np.mean(loss_te),np.mean(acc_tr), np.mean(acc_te)

In [27]:
def find_best_lambda(y,x, degree):
    seed = 12
    k_fold = 5
    lambdas = np.logspace(-4, 0, 40)
    best_lambda_tr = 0
    best_lambda_te = 0
    best_acc_tr = 0
    best_acc_te = 0
    val_l_tr = (0,0,0,0)
    val_l_te = (0,0,0,0)
    # split data in k fold
    k_indices = build_k_indices(y, k_fold, seed)
    # define lists to store the loss and accuracy of training data and test data
    rmse_tr = []
    rmse_te = []
    accuracy_tr = []
    accuracy_te = [] 
    # cross validation
    for lambda_ in lambdas:
        rmse_tr_tmp = []
        rmse_te_tmp = []
        acc_tr_tmp = []
        acc_te_tmp = []
        for k in range(k_fold-1):
            loss_tr, loss_te,acc_tr,acc_te = cross_validation_rg(y, x, k_indices, k+1, lambda_, degree)
            if(acc_tr>best_acc_tr):
                best_acc_tr = acc_tr
                best_lambda_tr = lambda_
                val_l_tr=(loss_tr,loss_te,acc_tr,acc_te)
            if(acc_te>best_acc_te):
                best_acc_te = acc_te
                best_lambda_te = lambda_
                val_l_te=(loss_tr,loss_te,acc_tr,acc_te)
            rmse_tr_tmp.append(loss_tr)
            rmse_te_tmp.append(loss_te)
            acc_tr_tmp.append(acc_tr)
            acc_te_tmp.append(acc_te)
            #print("rmse_tr_temp = ",rmse_tr_tmp)
        rmse_tr.append(np.mean(rmse_tr_tmp))
        rmse_te.append(np.mean(rmse_te_tmp))
        accuracy_tr.append(np.mean(acc_tr_tmp))
        accuracy_te.append(np.mean(acc_te_tmp))
    #cross_validation_visualization(lambdas, rmse_tr, rmse_te)
    return (best_lambda_te,val_l_te)  
    #return (lambdas, rmse_tr,rmse_te)



In [117]:
def find_best_overall_param():
    for deg in range(8):
        best_lambda1,loss_acc_1 = find_best_lambda(y_1,tx_1,deg)
        print("For batch " + str(1) +" and degree : " + str(deg) + " The best Test (lambda,(losstr,losste,acctr,accte)) : "+ str((best_lambda1,loss_acc_1)))
    
    for deg in range(8):
        best_lambda2,loss_acc_2 = find_best_lambda(y_2,tx_2,deg)
        print("For batch " + str(2) +" and degree : " + str(deg) + " The best Test (lambda,(losstr,losste,acctr,accte)) : "+ str((best_lambda2,loss_acc_2)))
    
    for deg in range(8):
        best_lambda3,loss_acc_3 = find_best_lambda(y_3,tx_3,deg)
        print("For batch " + str(3) +" and degree : " + str(deg) + " The best Test (lambda,(losstr,losste,acctr,accte)) : "+ str((best_lambda3,loss_acc_3)))
        
    return (best_lambda1,loss_acc_1),(best_lambda2,loss_acc_2),(best_lambda3,loss_acc_3)
    

In [118]:
(best_lambda1,loss_acc_1),(best_lambda2,loss_acc_2),(best_lambda3,loss_acc_3) = find_best_overall_param()

For batch 1 and degree : 0 The best Test (lambda,(losstr,losste,acctr,accte)) : (0.0001603718743751331, (0.30029315711432913, 0.30555205116348055, 0.7961821889700731, 0.7954283855469924))
For batch 1 and degree : 1 The best Test (lambda,(losstr,losste,acctr,accte)) : (0.0002030917620904735, (0.2388037549936008, 0.24187571795288057, 0.8330497447702931, 0.8323240916825143))
For batch 1 and degree : 2 The best Test (lambda,(losstr,losste,acctr,accte)) : (0.00012663801734674035, (0.2366960779033006, 0.23970574044351156, 0.8342664648183366, 0.8334000600540485))
For batch 1 and degree : 3 The best Test (lambda,(losstr,losste,acctr,accte)) : (0.0005223345074266843, (0.23388532876818713, 0.7602697967947677, 0.8362995445901311, 0.8357897107396658))
For batch 1 and degree : 4 The best Test (lambda,(losstr,losste,acctr,accte)) : (0.0001, (0.2315874496225282, 0.24314710727620556, 0.8395118106295666, 0.8387548793914523))
For batch 1 and degree : 5 The best Test (lambda,(losstr,losste,acctr,accte)) 

In [119]:
(best_lambda1,loss_acc_1),(best_lambda2,loss_acc_2),(best_lambda3,loss_acc_3)

((0.0004124626382901352,
  (0.22680842055372247,
   1.5701176846727192,
   0.8437155940346313,
   0.8428085276749074)),
 (0.008886238162743407,
  (0.30483956393233613,
   0.30271105268090254,
   0.7927682486458602,
   0.7959117874645344)),
 (0.00025719138090593444,
  (0.26835061634272156,
   0.271651836448498,
   0.8253205128205128,
   0.8298180314309347)))

In [121]:
def find_best_overall_param2():
    for deg in range(8,15):
        best_lambda1,loss_acc_1 = find_best_lambda(y_1,tx_1,deg)
        print("For batch " + str(1) +" and degree " + str(deg) + " The best Test (lambda,(losstr,losste,acctr,accte)) : "+ str((best_lambda1,loss_acc_1)))
    
    for deg in range(8,15):
        best_lambda2,loss_acc_2 = find_best_lambda(y_2,tx_2,deg)
        print("For batch " + str(2) +" and degree " + str(deg) + " The best Test (lambda,(losstr,losste,acctr,accte)) : "+ str((best_lambda2,loss_acc_2)))
    
    for deg in range(8,15):
        best_lambda3,loss_acc_3 = find_best_lambda(y_3,tx_3,deg)
        print("For batch " + str(3) +" and degree " + str(deg) + " The best Test (lambda,(losstr,losste,acctr,accte)) : "+ str((best_lambda3,loss_acc_3)))
        
    return (best_lambda1,loss_acc_1),(best_lambda2,loss_acc_2),(best_lambda3,loss_acc_3)

In [122]:
(best_lambda4,loss_acc_4),(best_lambda5,loss_acc_5),(best_lambda6,loss_acc_6) = find_best_overall_param2()

For batch 1 and degree 8 The best Test (lambda,(losstr,losste,acctr,accte)) : (0.0001603718743751331, (0.22460794609718232, 7.8884319405228185, 0.8457924632168952, 0.8455359823841457))
For batch 1 and degree 9 The best Test (lambda,(losstr,losste,acctr,accte)) : (0.0001603718743751331, (0.2239903660605052, 0.8375191665376706, 0.8464367931138024, 0.84618656791112))
For batch 1 and degree 10 The best Test (lambda,(losstr,losste,acctr,accte)) : (0.0001, (0.2236858067487893, 42.294082877087355, 0.8464868381543389, 0.8463367030327296))
For batch 1 and degree 11 The best Test (lambda,(losstr,losste,acctr,accte)) : (0.003455107294592218, (0.22677148719526288, 393516175822863.2, 0.8461959513562205, 0.845248223401061))
For batch 1 and degree 12 The best Test (lambda,(losstr,losste,acctr,accte)) : (0.0001603718743751331, (0.22300894328367965, 18898.77006311145, 0.8474189270343309, 0.8467370633570213))
For batch 1 and degree 13 The best Test (lambda,(losstr,losste,acctr,accte)) : (0.0001266380173

In [ ]:
#result for ridges regression
#1 lambda = 0.0001                 degree = 10  mean acc = 0.8467370633570213   std acc = 0.0007417655852458556 
#2 lambda = 0.0004124626382901352  degree = 12  mean acc = 0.808099045653856    std acc = 0.00018879961116990187
#3 lambda = 0.0008376776400682916  degree = 12  mean acc = 0.8451199338296113   std acc = 0.000357086967451429

# for submission :

In [30]:
tx_1= expand_with_pairwise_products(tx_1,12)

In [31]:
tx_2= expand_with_pairwise_products(tx_2,14)

In [32]:
tx_3= expand_with_pairwise_products(tx_3,13)

In [33]:
w1 , l1 = imp.ridge_regression(y_1,tx_1, 0.0001603718743751331)
w2 , l2 = imp.ridge_regression(y_2,tx_2,0.0001603718743751331)
w3 , l3 = imp.ridge_regression(y_3,tx_3,0.002154434690031882)

In [34]:
xtest_1 = expand_with_pairwise_products(xtest_1,12)

In [35]:
xtest_2 = expand_with_pairwise_products(xtest_2,14)

In [36]:
xtest_3 = expand_with_pairwise_products(xtest_3,13)

## Best parameters with logistic regression

In [55]:
def cross_validation_lg(y, x, degree):
    gamma = 0.001
    max_iter = 2000
    k_fold = 5 
    seed = 12
    initial_w = np.zeros((x.shape[1], 1))
    k_indices = build_k_indices(y, k_fold, seed)
    loss_te = np.array([])
    loss_tr = np.array([])
    acc_te = np.array([]) 
    acc_tr = np.array([]) 
    #print(x.shape)
    for i in range(k_fold):
        sub = np.delete(k_indices,i,axis=0).flatten()
        x_train = x[sub]
        y_train = y[sub]
        x_test = x[k_indices[i]]
        y_test = y[k_indices[i]]

        #print('train expanded',x_train.shape)
        #print('test expanded',x_test.shape)
        
        x_train = expand_with_pairwise_products(x_train,degree)
        x_test = expand_with_pairwise_products(x_test,degree)
        #x_train = np.append(x_train,np.exp(x_train),axis=1)
        
        #x_train = np.append(x_train,np.cos(x_train),axis=1)
        #x_train = np.append(x_train,np.sin(x_train),axis=1)
        #x_train = add_bias(x_train)
        #x_test = np.append(x_test,np.exp(x_test),axis=1)
        #x_test = np.append(x_test,np.cos(x_test),axis=1)
        #x_test = np.append(x_test,np.sin(x_test),axis=1)
        
        #x_test = add_bias(x_test)


        #w,loss_train = imp.least_squares(y_train, x_train)
        w,loss_train = imp.logistic_regression(y_train, x_train,initial_w,max_iter,gamma) 
        acc_train = compute_accuracy(y_train,x_train,w)
        acc_test = compute_accuracy(y_test,x_test,w)
        #print(loss_train)
        loss_test = imp.calculate_loss(w, x_test, y_test)
        loss_tr = np.append(loss_tr,loss_train)
        loss_te= np.append(loss_te,loss_test)
        acc_tr = np.append(acc_tr,acc_train)
        acc_te = np.append(acc_te,acc_test)
    
    return degree, (np.mean(loss_tr),np.mean(loss_te),np.mean(acc_tr), np.mean(acc_te))

In [56]:
def find_best_degree():
    for deg in range(8):
        degree1,loss_acc_1 = cross_validation_lg(y_1,tx_1, deg)
        print("For batch " + str(1) +" and degree : " + str(deg) + " The best Test (lambda,(losstr,losste,acctr,accte)) : "+ str((best_lambda1,loss_acc_1)))
    
    for deg in range(8):
        degree2,loss_acc_2 = cross_validation_lg(y_2,tx_2,deg)
        print("For batch " + str(2) +" and degree : " + str(deg) + " The best Test (lambda,(losstr,losste,acctr,accte)) : "+ str((best_lambda2,loss_acc_2)))
    
    for deg in range(8):
        degree3,loss_acc_2 = cross_validation_lg(y_3,tx_3,deg)
        print("For batch " + str(3) +" and degree : " + str(deg) + " The best Test (lambda,(losstr,losste,acctr,accte)) : "+ str((best_lambda3,loss_acc_3)))
        
    return (degree1,loss_acc_1),(degree2,loss_acc_2),(degree3,loss_acc_2)
    

In [57]:
(degree1,loss_acc_1),(degree2,loss_acc_2),(degree3,loss_acc_2) = find_best_degree()

ValueError: shapes (79928,121) and (16,1) not aligned: 121 (dim 1) != 16 (dim 0)

## Generate predictions and save ouput in csv format for submission:

In [117]:
y_pred1 = predict_labels(w1, xtest_1)
y_pred2 = predict_labels(w2, xtest_2)
y_pred3 = predict_labels(w3, xtest_3)

In [123]:
w1

array([[nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
       [nan],
      

In [118]:
y = np.concatenate((y_pred1,y_pred2))
y = np.concatenate((y,y_pred3))

In [119]:
ids = np.concatenate((ids_1,ids_2))
ids = np.concatenate((ids,ids_3))

In [120]:
OUTPUT_PATH = '../resources/gd.csv'
create_csv_submission(ids, y,OUTPUT_PATH)

ValueError: cannot convert float NaN to integer